In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

csv_path = "exercitii_extrase_structurat_rezolvari.csv"
df = pd.read_csv(csv_path)

# Pastreaz doar r0andurile care au o solutie
df = df[df["solutie"].notna() & (df["solutie"].str.strip() != "")]

# Prompt user (ID + cerință + variante a–f)
def build_prompt(row):
    question = f"{row['id']}. {row['intrebare']}\n"
    choices = [
        f"{opt}) {str(row[opt]).strip()}"
        for opt in ['a', 'b', 'c', 'd', 'e', 'f']
        if pd.notna(row[opt]) and str(row[opt]).strip() != ""
    ]
    return question + "\n " + ";\n ".join(choices)

# Prompt asistent
def build_assistant_response(row):
    parts = [f"Răspuns corect: {row['raspuns_corect']}"]
    if pd.notna(row["indicatii"]) and row["indicatii"].strip() != "":
        parts.append(f"Indicații: {row['indicatii'].strip()}")
    if pd.notna(row["solutie"]) and row["solutie"].strip() != "":
        parts.append(f"Soluție: {row['solutie'].strip()}")
    return "\n ".join(parts)

# Structura completa pentru fine-tuning
def build_message(row):
    return {
        "messages": [
            {
                "role": "system",
                "content": "Ești un asistent care rezolvă exerciții de matematică pentru admitere și oferă explicații clare."
            },
            {
                "role": "user",
                "content": build_prompt(row)
            },
            {
                "role": "assistant",
                "content": build_assistant_response(row)
            }
        ]
    }

# Construieste datele si imparte in train/test
data = [build_message(row) for _, row in df.iterrows()]
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Salveaza in format JSONL
def save_jsonl(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        for item in data:
            json.dump(item, f, ensure_ascii=False)
            f.write("\n")

save_jsonl(train_data, "fine_tune_train.jsonl")
save_jsonl(test_data, "fine_tune_test.jsonl")

print("Fișierele au fost salvate cu succes.")

✅ Fișierele au fost salvate cu succes.
